# 챗봇 구축
## 개요

LLM 기반 챗봇을 설계하고 구현하는 방법의 예를 살펴보겠습니다. 이 챗봇은 대화를 나누고 이전 상호 작용을 기억할 수 있습니다.
우리가 구축한 이 챗봇은 언어 모델만 사용하여 대화를 나눕니다. 다음과 같은 몇 가지 다른 관련 개념을 찾을 수 있습니다.

- [대화형 RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/): 외부 데이터 소스를 통해 챗봇 경험을 활성화합니다.
- [챗봇](https://python.langchain.com/v0.2/docs/tutorials/agents/): 조치를 취할 수 있는 챗봇 구축

## 개념

우리가 작업할 고급 구성 요소입니다.

- [채팅 모델](https://python.langchain.com/v0.2/docs/concepts/#chat-models): 챗봇 인터페이스는 원시 텍스트가 아닌 메시지를 기반으로 하므로 텍스트 LLM보다는 채팅 모델에 가장 적합합니다.
- [프롬프트 템플릿](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates): 기본 메시지, 사용자 입력, 채팅 기록 및 검색된 추가 컨텍스트를 결합하는 프롬프트를 조합하는 프로세스를 단순화 합니다.
- [채팅 기록](https://python.langchain.com/v0.2/docs/concepts/#chat-history): 챗봇이 과거 상호작용을 기억하고 후속 질문에 응답할 때 이를 고려할 수 있습니다.
- [LangSmith](https://python.langchain.com/v0.2/docs/concepts/#langsmith) 를 사용하여 응용 프로그램 디버깅 및 추적
  
위의 구성 요솔를 함께 사용하여 강력한 대화형 챗봇을 만드는 방법을 다룹니다.

## 설치

### Jupyther Notebook

이 예제는 Jupyter Notebook을 사용하여 진행합니다.


### 라이브러리 설치

In [4]:
pip install langchain python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


### 환경 변수 설정 및 랭스미스

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2") # true
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") # your langchain api key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_LLM")
embedding_model_name = os.getenv("OPENAI_EMBEDDING_MODEL")

### LLM 설치

In [6]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


## 개발

먼저 LLM 모델 인스턴스를 불러옵니다.

모델에 Messages를 입력해주고 invoke하여 모델을 호출해줍니다.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 생성
model = ChatOpenAI(model=model_name)

model.invoke([HumanMessage(content="Hi! I'm Jaehyun")])

# AIMessage(content='Hello Jaehyun! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cff4f501-04dd-4d5f-8852-d5d274b37948-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26})

AIMessage(content='Hello Jaehyun! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cff4f501-04dd-4d5f-8852-d5d274b37948-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26})

모델 자체에는 상태 개념이 없습니다. 후속 질문을 하는 경우 이전의 질문 내용을 LLM이 저장하고 있지 않습니다.

In [3]:
model.invoke([HumanMessage(content="What's my name?")])

# AIMessage(content='I am sorry, but I do not have the ability to know your name.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 12, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b09691d3-8400-4e13-9301-9fc47f2375c7-0', usage_metadata={'input_tokens': 12, 'output_tokens': 16, 'total_tokens': 28})

AIMessage(content='I am sorry, but I do not have the ability to know your name.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 12, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b09691d3-8400-4e13-9301-9fc47f2375c7-0', usage_metadata={'input_tokens': 12, 'output_tokens': 16, 'total_tokens': 28})

[LangSmith 추적](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r) 예제를 살펴보면
이전 대화가 맥락으로 바뀌지 않아 질문에 답할 수 없다는 것을 알 수 있습니다. 이 문제를 해결하려면 전체 대화 기록을 모델에게 전달해야합니다.

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Jaehyun"),
        AIMessage(content="Hello Jaehyun! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

# AIMessage(content='Your name is Jaehyun.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 39, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-796d88b4-7558-4e82-9c7d-37836d1ff9dc-0', usage_metadata={'input_tokens': 39, 'output_tokens': 7, 'total_tokens': 46})

AIMessage(content='Your name is Jaehyun.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 39, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-796d88b4-7558-4e82-9c7d-37836d1ff9dc-0', usage_metadata={'input_tokens': 39, 'output_tokens': 7, 'total_tokens': 46})

이렇게 대화 식으로 구현하면 챗봇이 이전 대화 내용을 토대로 상호작용할 수 있습니다. 그렇다면 이를 가장 잘 구현하려면 어떻게 해야하는지 알아봅시다

### 메시지내역

Message History 클래스를 사용하여 모델을 래핑하고 상태 저장으로 만들 수 있습니다. 이렇게 하면 모델의 입력과 출력을 추적하고 일부 데이터 저장소에 저장합니다. 그런 다음 향후 상호작용은 해당 메시지를 로드하고 입력의 일부로 체인에 전달합니다.

#### langchain-community 설치

In [5]:
pip install langchain_community

  Using cached langchain_community-0.2.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached dataclasses_json-0.6.6-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.21.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.1-py3-none-any.whl (2.1 MB)
Using cached dataclasses_json-0.6.6-py3-none-any.whl (28 kB)
Using cached marshmallow-3.21.2-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


#### 구현

`langchain_community`를 설치하였으면 관련 클래스를 가져오고 모델을 래핑해 메시지 기록을 추가하는 체인을 만듭니다.

여기서 중요한 점은 이 함수는 메시지 기록을 개체(session)를 가져와서 반환해야합니다. 별도의 대화를 구별하는 데 사용되며 새 체인을 호출할 때 구성의 일부로 전달되어야합니다.

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [7]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke([HumanMessage(content="Hi! I.m work in infocz")], config=config)

print(f"{response.content}") # That's great to hear! What kind of work do you do at Infocz?

That's great to hear! What kind of work do you do at Infocz?


In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Where do i work?")],
    config=config,
)

print(f"{response.content}")

# I apologize for the confusion. You mentioned that you work at Infocz in your previous message. Can you please clarify what kind of work you do at Infocz?

I apologize for the confusion. You mentioned that you work at Infocz in your previous message. Can you please clarify what kind of work you do at Infocz?


챗봇이 이전의 대화를 기억하고 있습니다. 다른 `session_id`를 참조하면 대화가 새로 시작됩니다.

In [9]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Where do i work?")],
    config=config,
)

print(f"{response.content}")

# I'm a virtual assistant and do not have a physical location where I work. I exist in the digital world to assist with tasks and answer questions.

I'm a virtual assistant and do not have a physical location where I work. I exist in the digital world to assist with tasks and answer questions.


### 프롬프트 템플릿

프롬프트 템플릿은 원시 사용자 정보를 LLM이 작업할 수 있는 포맷으로 변환하는데 도움이 됩니다.
이 경우 원시 사용자 입력은 LLM에 전달하는 메시지일 뿐입니다.
몇 가지 사용자 지정 지침이 포함된 시스템 메시지를 추가해 봅시다.

시스템 메시지를 추가하기 위해 ChatPromptTemplate를 만듭니다.

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Answer all questions to the best of your ability"
    ),
    MessagesPlaceholder(variable_name="messages"),
  ]
)

chain = prompt | model

`MessagesPlaceholder(variable_name="messages")`는 입력 유형을 약간 변형한 것입니다. 메시지 목록이 포함된 키(`messages`)가 있는 딕셔너리를 전달합니다.

In [12]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Jaehyun")]})

print(f"{response.content}")

Hello Jaehyun! How can I assist you today?


아까 전 메시지 기록 개체 ( `RunnableWithMessageHistory` )로 해당 체인을 래핑합니다.

In [13]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": { "session_id" : "abc5" }}

response = with_message_history.invoke(
  [HumanMessage(content="Hi! I'm Jaehyun")],
  config=config
)

print(f"{response.content}")

response = with_message_history.invoke(
  [HumanMessage(content="What's my name?")],
  config=config
)

print(f"{response.content}")

Hello Jaehyun! How can I assist you today?
Your name is Jaehyun.


더 복잡한 프롬프트 템플릿을 만들어 보겠습니다. 답변 내용을 특정 언어로 번역하여 반환하고자 합니다. 그러기 위해서는 시스템 메시지 내에 특정 언어를 변수로 설정해야합니다.

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Jaehyun")], "language": "korean"},
    config=config,
)

print(f"{response.content}")

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="what's my name?")], "language": "korean"},
    config=config,
)

print(f"{response.content}")

안녕하세요, 재현님! 만나서 반가워요. 어떻게 도와드릴까요?
당신의 이름은 Jaehyun입니다.


### 대화 기록 관리

챗봇을 구축할 때 중요한 개념 중 하나는 대화 기록을 관리하는 방법입니다. 관리되지 않는 상태로 두면 메시지 목록이 무제한으로 증가하여 LLM의 컨텍스트 창을 오버플로, 즉 토큰 제한에 걸릴 수 있습니다. 따라서 전달하는 메시지 크기를 제한하는 단계를 추가하는 것이 중요합니다.

**중요한 것은 프롬프트 템플릿을 사용하기 전에 이 작업을 수행하지만 메시지 기록에서 이전 메시지를 로드한 후에 수행하려고 한다는 것입니다.**

키를 적절하게 수정하는 프롬프트 앞에 간단한 단계를 추가한 다음 메시지 기록 클래스에서 새 체인을 래핑하여 이 작업을 수행합니다.

전달된 메시지를 수정할 함수를 정의합니다.

가장 최근 메시지를 선택하다로고 만들겠습니다.

In [16]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
  return messages[-k:]

chain = (
  RunnablePassthrough.assign(messages= lambda x: filter_messages(x["messages"]))
  | prompt
  | model
)

앞서 `prompt`의 `MessagesPlaceholder(variable_name="messages")`를 봅시다. messages가 변수명으로 지정되어 있습니다.
`RunnablePassthrough` 프롬프트 템플릿에 지정된 변수 즉 딕셔너리 키의 값을 컨트롤할 수 있습니다.
`RunnablePassthrough.assign(messages=)`를 통해 messages 키의 메시지 목록 값을 가져와 `lambda x: filter_messages(x["messages"])`함수의 매개변수로 넣어주고 반환된 값을 messages의 값으로 활용합니다.

In [20]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

10개 이상의 메시지 길이의 메시지 목록을 만들면 초기 메시지에서 이름에 대한 정보를 기억하지 못하는 상황을 볼 수 있습니다.
그러나 마지막 10개 내에 있는 정보 ( 아이스크림 맛 ) 를 물어보면 여전히 기억하고 있습니다


In [18]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "Korean",
    }
)
print(f"{response.content}")

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "Korean",
    }
)
print(f"{response.content}")

I'm sorry, I don't know your name.
바닐라 아이스크림입니다.


이제 아까 처럼 메시지 기록에 해당 체인을 래핑해 봅시다.

In [21]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "Korean",
    },
    config=config,
)

print(f"{response.content}")

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what's my favorite ice cream?")],
        "language": "Korean",
    },
    config=config,
)

print(f"{response.content}")

I'm sorry, I don't have access to personal information.
I'm sorry, I don't have access to that information.


이름을 물으면서 ai 답변하는 것까지 총 2개의 채팅이 신규 추가되었습니다. 따라서 가장 좋아하는 아이스크림 맛에 대한 채팅 기록이 최근 10개의 메시지 목록 내에 존재하지 않게 되면서 ai는 모른다고 답변을 합니다.

### 스트리밍

챗봇을 개발했습니다. 그러나 챗봇 애플리케이션에서 정말 중요한 UX 고려 사항 중 하나는 스트리밍입니다.
LLM은 때때로 응답하는데 시간이 오래 걸릴 수 있으므로 사용자 경험을 개선하기 위해 대부분의 애플리케이션이 수행하는 한 가지는 각 토큰이 생성될 때 다시 스트리밍 하는 것 입니다.
이를 통해 사용자는 진행 상황을 볼 수 있습니다.

`.stream`을 사용하여 스트리밍 응답을 가져올 수 있습니다.

In [23]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="안녕! 나는 재현이야. 내게 재밌는 농담을 들려줄래?")],
        "language": "Korean",
    },
    config=config,
):
    print(r.content, end="|")

|안|녕|하세요|,| 재|현|님|!| 물|론|이|죠|.| 여|기| 재|미|있|는| 농|담|이| 있|어|요|.

|"|왕|이| 꿈|을| 꾸|었|어|요|.| 어|떤| 꿈|이|었|을|까|요|?| 바|로| 왕|궁|에서| 왕|이| 잠|을| 자|는| 꿈|이|었|어|요|!"

|재|밌|고| 유|머|러|스|한| 농|담|이|었|나|요|?| 더| 들|려|드|릴|게| 있|으면| 언|제|든|지| 말|씀|해|주세요|!||